# Install & Setup



In [7]:
!pip install -qU langchain langchain-community langchain-huggingface langchain-chroma chromadb pypdf sentence-transformers transformers accelerate --quiet
print("Dependencies installed and imported quietly!")

Dependencies installed and imported quietly!


In [9]:
!pip install --force-reinstall -qU langchain langchain-community langchain-huggingface langchain-chroma chromadb pypdf sentence-transformers transformers accelerate --quiet
print("Dependencies installed and imported quietly!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [10]:
import re
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA
from transformers import pipeline

# Loading the PDF

Uploading the Document

In [11]:
print("Please upload your PDF:")
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]

loader = PyPDFLoader(pdf_filename)
raw_docs = loader.load()

Please upload your PDF:


Saving Machine Learning For Absolute Beginners.pdf to Machine Learning For Absolute Beginners.pdf


Cleaning the docs

In [13]:
for doc in raw_docs:
    doc.page_content = re.sub(r'\s+', ' ', doc.page_content)
    doc.metadata["source"] = pdf_filename

In [15]:
print(f"Loaded and cleaned {len(raw_docs)} pages from {pdf_filename}.")

Loaded and cleaned 128 pages from Machine Learning For Absolute Beginners.pdf.


# Chunking

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [17]:
chunks = text_splitter.split_documents(raw_docs)

In [18]:
print(f"Sliced into {len(chunks)} chunks.")

Sliced into 374 chunks.


# Embedding

In [19]:
print("Loading Embedding Model...")
# Turns text into math numbers so the computer can search it
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Loading Embedding Model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Vector DB - Chroma

In [20]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="my_pdf_collection"
)

# LLM

In [21]:
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_new_tokens=512,
    device=0 # '0' ensures it uses the fast Colab GPU
)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [22]:
llm = HuggingFacePipeline(pipeline=pipe)
print("LLM loaded and ready!")

LLM loaded and ready!


In [34]:
test_question = "What is the capital of France?"
print(f"Asking: '{test_question}'")

Asking: 'What is the capital of France?'


In [35]:
test_answer = llm.invoke(test_question)

print(f"\n AI Answered: {test_answer}")


 AI Answered: paris


# Retrieval

Metadata filtering

In [28]:
search_filter = {"source": pdf_filename}
print(f"Metadata filter created for: {search_filter['source']}")

Metadata filter created for: Machine Learning For Absolute Beginners.pdf


In [33]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 4, # Fetch the top 4 most relevant chunks
        "filter": search_filter
    }
)

# Prompt engineering

In [36]:
prompt_template = """
You are a helpful and professional assistant.
Answer the question based ONLY on the following context.
If the answer is not in the context, clearly state "I cannot find the answer in the provided document."

Context: {context}

Question: {question}

Helpful Answer:"""

QA_PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print("Prompt template crafted and ready.")

Prompt template crafted and ready.


# Q&A with our model

In [47]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the chain for generating the answer
rag_chain_from_docs = (
    QA_PROMPT
    | llm
    | StrOutputParser()
)

# Define the full RAG chain including retrieval and source documents using LCEL
qa_chain = (
    {
        "context": itemgetter("question") | retriever | format_docs,
        "question": RunnablePassthrough(),
        "source_documents": itemgetter("question") | retriever
    }
    | RunnablePassthrough.assign(answer=rag_chain_from_docs)
)

print("System Ready!\n")

System Ready!



In [48]:
question = input("Ask a question about your PDF: ")

print("Thinking...")
response = qa_chain.invoke({"question": question})

# Print the clean output
print("\n---AI RESPONSE ---")
print(response["answer"])

print("\n---RETRIEVED SOURCES ---")
for i, doc in enumerate(response["source_documents"]):
    # We use the metadata we saved way back in Cell 2!
    print(f"Source {i+1} (From {doc.metadata.get('source', 'unknown')}): {doc.page_content[:150]}...")

Ask a question about your PDF: what is supervised learning
Thinking...

---AI RESPONSE ---
Supervised learning concentrates on learning patterns through connecting the relationship between variables and known outcomes and working with labeled datasets.

---RETRIEVED SOURCES ---
Source 1 (From Machine Learning For Absolute Beginners.pdf): . These three categories are supervised , unsupervised, and reinforcement . Supervised Learning As the first branch of machine learning, supervised le...
Source 2 (From Machine Learning For Absolute Beginners.pdf): . The fact that the output and feature values are known qualifies the dataset as “labeled.” The algorithm then deciphers patterns that exist in the da...
Source 3 (From Machine Learning For Absolute Beginners.pdf): machine learning. Unlike supervised and unsupervised learning, reinforcement learning continuously improves its model by leveraging feedback from prev...
Source 4 (From Machine Learning For Absolute Beginners.pdf): . For those who